# Creando la API

Una **API** es una interfaz de programación de aplicaciones (en inglés *Application Programming Interface*). Esencialmente permiten comunicar programas creados con diferentes arquitecturas entre ellos. 

La nuestra será una API web que aceptará cuatro métodos de petición HTTP esenciales:

* `GET`: Para consultar clientes.
* `POST`: Para crear clientes.
* `PUT`: Para actualizar clientes.
* `DELETE`: Para borrar clientes.

Existen otros métodos, si queréis más información los encontraréis [en la Wikipedia](https://es.wikipedia.org/wiki/Protocolo_de_transferencia_de_hipertexto#Métodos_de_petición).

## Primera respuesta

Las respuestas se definen en funciones asíncronas decoradas con el método deseado.

Nuestra primera respuesta será de de tipo `GET`, configurada en la raíz `/` y devolverá un `JSONResponse` con el contenido de un diccionario:

```python
from fastapi import FastAPI
from fastapi.responses import JSONResponse

app = FastAPI()

@app.get("/")
async def index():
    content = {'mensaje': '¡Hola mundo!'}
    return JSONResponse(content=content)

print("Servidor de la API...")
```

Notaréis que si ejecutamos el script no se queda en marcha:

```bash
> pipenv run python api.py
```

Esto se debe a que la API es un servicio definido explícitamente en la variable `app` , debemos ponerlo en marcha mediante un servidor web ASGI como [uvicorn](https://github.com/encode/uvicorn).


## Sirviendo la API

El comando es el siguiente:

```bash
> pipenv run uvicorn api:app --reload
```

El argumento `--reload` permitirá que `uvicorn` detecte los cambios en el fichero y reinicie el servicio automáticamente.

Si consideráis como yo que el comando es demasiado largo, podemos crear un script en `Pipenv`:

```toml
[scripts]
api = "uvicorn api:app --reload"
```

Al ejecutarlo:

```bash
> pipenv run api
```

## Primera petición

Se iniciará el servidor en `127.0.0.1` y el puerto `8000`, si accedemos a la dirección http://127.0.0.1:8000 se hará automáticamente una petición `GET` y nos dará la respuesta que hemos programado:

```json
{"mensaje":"Â¡Hola mundo!"}
```
Como vemos el contenido lo devuelve sin codificación, si queremos poner alguna podemos establecer unas cabeceras que podemos crear globalmente para reutilizarlas:

```python
headers = {"content-type": "charset=utf-8"}
    
@app.get("/")
async def index():
    content = {'mensaje': '¡Hola mundo!'}
    return JSONResponse(content=content, headers=headers)
```

Los símbolos unicode ahora se verán correctamente:

```json
{"mensaje":"¡Hola mundo!"}
```

## Documentación automática

`FastAPI` que genera una documentación en http://127.0.0.1:8000/docs con un cliente para pruebas:

![](docs/img02.png)

Como bien nos indican también podemos realizar la petición mediante `cURL` en una terminal:

In [ ]:
!curl -X "GET" "http://127.0.0.1:8000/" -H "accept: application/json"

## Códigos de respuesta

Es importante notar que la respuesta contiene un código `200`, eso significa que ha sido exitosa. Los códigos más comunes del protocolo HTTP son:

* **1XX Respuestas informativas**: `100` Continue, `103` Checkpoint...
* **2XX Respuestas correctas**: `200` OK, `201` Created...
* **3XX Redirecciones**: `301` Moved Permanently, `302` Moved Temporarily...
* **4XX Errores del cliente**: `400` Bad Request, `401` Authorization Required, `403` Forbidden, `404` Not Found...
* **5XX Errores del servidor**: `500` Internal Server Error, `503` Service Temporarily Unavaible...


La lista completa podéis encontrarla [en la Wikipedia](https://es.wikipedia.org/wiki/Anexo:Códigos_de_estado_HTTP).

## Media Type

Este es otro campo que debemos comentar pues hace referencia al tipo de la respuesta. 

Por defecto el objeto `JSONResponse` devuelve una cadena `JSON` cuyo *media type* es `application/json`, pero puede ser de cualquier otro dependiendo de lo que queramos responder.

Por ejemplo podemos crear una respuesta en crudo en la ruta `/html/` de tipo `GET` para devolver una cadena con código HTML:

```python
from fastapi import FastAPI, Response

@app.get("/html/")
def html():
    content = """
    <!DOCTYPE html>
    <html lang="es">
    <head>
        <meta charset="UTF-8">
        <title>¡Hola mundo!</title>
    </head>
    <body>
        <h1>¡Hola mundo!</h1>
    </body>
    </html>
    """
    return Response(content=content, media_type="text/html")
```

Si accedemos a la dirección http://127.0.0.1:8000/html/ el navegador interpretará el fichero directamente:

![](docs/img03.png)

La lista completa de *media types* se puede encontrar [en la Wikipedia](https://en.wikipedia.org/wiki/Media_type).